In [1]:
import torch
import torchvision
from torch import nn
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image 
import numpy as np
import dataset_utils
import importlib
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier 
from sklearn.preprocessing import StandardScaler

importlib.reload(dataset_utils)

<module 'dataset_utils' from '/home/fogliodicarta/Desktop/InsectClassification/dataset_utils.py'>

In [8]:
csv_path='final_dataset.csv'
image_path = 'image_dataset/'
batch_size = 1000
shuffle_loaders = False

In [9]:
from dataset_utils import *

In [20]:
df = pd.read_csv(csv_path,index_col=0)
imsize = 64
tform = transforms.Compose([transforms.Resize((imsize,imsize)),
                            transforms.PILToTensor(),
                            transforms.ConvertImageDtype(torch.float),
                            transforms.Normalize(0.5,0.5)])
image_dataset = torchvision.datasets.ImageFolder(image_path, transform=tform)

img2dna = get_imgs_bold_id(image_dataset,df)

nucleotides = df[['nucleotide','species_name','genus_name','processid','image_urls']]
colonna_dna = df.loc[:,"nucleotide"]
nucleotides.loc[:,'nucleotide'] = colonna_dna.apply(one_hot_encoding)
random.seed(42)
X_train_val, X_test, y_train_val, y_test = data_split(nucleotides,0.2,random_state=42)
train_data = X_train_val
train_data['species_name'] = y_train_val
X_train, X_validation, y_train, y_validation = data_split(train_data,0.2,drop_labels=False,random_state=42)
train_indices, val_indices, test_indices = image_splits_from_df(X_train,X_validation,X_test,image_dataset)
 
class WholeDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.targets = data.targets            
    def __getitem__(self, index):
        x = self.data[index][0]
        y = self.targets[index]
        return x, y
    
    def __len__(self):
        return len(self.data)
        
whole_dataset = WholeDataset(image_dataset)
n_classes = np.unique(whole_dataset.targets).shape[0]
train_imgs = torch.utils.data.Subset(whole_dataset, train_indices)
val_imgs = torch.utils.data.Subset(whole_dataset, val_indices)
train_val_imgs = torch.utils.data.Subset(whole_dataset, train_indices+val_indices)
test_imgs = torch.utils.data.Subset(whole_dataset, test_indices)

train_loader = torch.utils.data.DataLoader(train_imgs, batch_size=len(train_imgs),shuffle=shuffle_loaders, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_imgs, batch_size=len(val_imgs),shuffle=shuffle_loaders, num_workers=2)
train_val_loader = torch.utils.data.DataLoader(train_val_imgs, batch_size=len(test_imgs),shuffle=shuffle_loaders, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_imgs, batch_size=batch_size,shuffle=shuffle_loaders, num_workers=2)
dataloaders = {"train":train_loader,"val":val_loader,"test":test_loader,'train_val':train_val_loader}
dataset_sizes = {'train': len(train_imgs.indices), 'val':len(val_imgs.indices),'test':len(test_imgs.indices),'train_val':len(train_val_imgs.indices)}

described_species_labels = np.array([image_dataset.targets[i] for i in train_indices])
described_species_labels = np.unique(described_species_labels)


In [21]:
described_species_labels_train = set(np.array([image_dataset.targets[i] for i in train_indices]))
described_species_labels_trainval = set(np.array([image_dataset.targets[i] for i in train_indices+val_indices]))

In [16]:
train_loc = train_indices
trainval_loc = train_indices+val_indices

n_seen_val = 0
val_all_loc = set()
val_seen_loc = set() 
for i in val_indices:
    val_all_loc.add(i)
    if image_dataset[i][1] in described_species_labels_train:
        n_seen_val+=1
        val_seen_loc.add(i)
val_unseen_loc = val_all_loc - val_seen_loc

n_seen_test = 0
test_all_loc = set()
test_seen_loc = set() 
for i in test_indices:
    test_all_loc.add(i)
    if image_dataset[i][1] in described_species_labels_trainval:
        n_seen_test+=1
        test_seen_loc.add(i)
test_unseen_loc = test_all_loc - test_seen_loc

In [25]:
tt = None
for x in train_loader:
    tt = (x)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x738a97786f20>
Traceback (most recent call last):
  File "/home/fogliodicarta/miniconda3/envs/insetti/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/fogliodicarta/miniconda3/envs/insetti/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/home/fogliodicarta/miniconda3/envs/insetti/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Traceback (most recent call last):
  File "/home/fogliodicarta/miniconda3/envs/insetti/lib/python3.12/multiprocessing/queues.py", line 259, in _feed
    reader_close()
  File "/home/fogliodicarta/miniconda3/envs/insetti/lib/python3.12/multiprocessing/connection.py", line 178, in clo

In [32]:
tt[0].shape

torch.Size([13039, 3, 64, 64])